In [2]:
import pandas as pd
import os 
import sqlite3
import streamlit as st

path_db = "banco_de_dados.db"

conn = sqlite3.connect(path_db)
df_cliente = pd.read_sql_query("SELECT * FROM clientes", conn) 
df_conta = pd.read_sql_query("SELECT * FROM contas", conn)

In [3]:
# Corrigindo as 3 primeiras linhas do Cpf para ficar no formato padrão (placeholder = 000.000.000-00)
    # Aqui defini uma lista para guardar as infos de como quero que fique os três primeiros cpf
novo_cpf = ["615.019.033-14" , "123.456.789-10","013.179.063.30"]
cpf_alvo = df_cliente['cpf'].head(3).tolist()
# Loop para percorrer e atualizar
for antigo, novo in zip(cpf_alvo, novo_cpf) : 
    df_cliente.loc[df_cliente['cpf']==antigo, 'cpf'] = novo

    df_conta.loc[df_conta['cpf_cliente'] == antigo, 'cpf_cliente'] = novo
# Junção das tuas tabelas estabelecidas no CRUD e refino na diferença de colunas (tabela.conta = cpf_cliente) e (tabela.cliente = cpf)
df_completo = pd.merge(df_conta, df_cliente, left_on="cpf_cliente", right_on="cpf")

df_completo = df_completo.drop(columns=["cpf_cliente"])

display(df_completo.head(5))

,id_x,numero,saldo,id_y,nome,cpf
0,1,34828,10980.86,1,Bruno Raphael Andrade Santos,615.019.033-14
1,2,63594,2236.49,2,Maria Victoria,123.456.789-10
2,3,5276,5373.46,3,Pedro Miguel,013.179.063.30
3,4,21717,4301.14,4,Kaique Pires,620.983.145-15
4,5,13005,1421.19,5,Brayan Duarte,269.451.730-70


In [ ]:
# Conferindo quais os tipos de dados temos aqui e extraindo algumas informações relevantes
def extracao_info (df_completo) : 
    col1, col2 = st.columns(2)
    with col1 : 
        st.metric (label = "Total de Clientes" , value= df_completo.shape[0])
    with col2 : 
        st.metric (label = "Total de colunas", value= df_completo.shape[1])

    st.divider()

    st.subheader("📊 Resumo Estatístico dos Saldos")
    metricas = df_completo.describe()

    c1,c2,c3 = st.columns(3)
    c1.metric ("Saldo  Médio", f"R$ {metricas.loc['mean', 'saldo']:2f}")
    c2.metric ("Saldo  Mínimo", f"R$ {metricas.loc['min', 'saldo']:2f}")
    c3.metric ("Saldo  Máximo", f"R$ {metricas.loc['max', 'saldo']:2f}")




# PERGUNTAS DE NEGÓCIO 

### Agora que ja temos o dataset limpo e tratado, posso começar a analisar e responder tais perguntas : 

-> Qual é o Valor Total sob Gestão (AUM - Assets Under Management)

-> Existe concentração de capital?

-> Como os clientes estão distribuídos por faixa de saldo?

## ​💲​ Qual é o valor total sob gestão do banco

In [ ]:
import streamlit as st
## -> Qual é o valor total sob gestão do banco

def Aum (df) : 
    tot = df['saldo'].sum()
    return st.metric (label = " O Assetes Under Management (AUM)", value =f"{tot:.2f}", help= "Soma total de todos os saldos de clientes no banco")





 O Assets Under Management (AUM) gira em torno de R$ 2544639.74


## 💰​ Existe concentração de capital

In [ ]:
import streamlit as st
import plotly as px 



## -> Existe concentração de capital ?
def concentracao (df) : 
    df_ordenado_saldo = df.sort_values(by = 'saldo', ascending=False)
    aum_tot = df['saldo'].sum()

    top_10_percent = int(len(df_ordenado_saldo )* (0.1))
    saldo_top_10 = df_ordenado_saldo.iloc[:top_10_percent]['saldo'].sum()
    saldo_restante = aum_tot - saldo_top_10
     
    porcentagem = (saldo_top_10 / aum_tot ) * 100
    st.subheader (" 💰 Existe Concentração de capital no banco ?")
    
    # Construção da figura 
    dados_grafico = {
        "Grupo" : ["Top 10 % Clientes" , "Restante (90%)"], "Valor" : [saldo_top_10, saldo_restante]
    }
    fig = px.pie (
        dados_grafico, values = 'Valor', names = 'Grupo', hole = 0.5, color_discrete_sequence = ['#00CC96', '#636EFA']
    )

    fig.update_traces(textposition = 'inside', textinfo ='percent+label')

    st.plotly_chart (fig, use_container_width=True) 
    st.info (f"Os 10 % melhores clientes detêm ** {porcentagem:.2f} ** do capital total ")

    
# RESPOSTA : Como o número de ficou em 19 %, é possível afirmar que a capital total do banco é bem distribuido, pois, os 10% maiores clientes so detêm 19% do tot.


Os 10% maiores cliente detêm 19.438940 %  do capital total (AUM) 


## ⭐​ Como os clientes estão distribuídos por faixa de saldo ?

In [11]:
import matplotlib.pyplot as plt 
import seaborn as sns
import streamlit as st
%matplotlib inline

bins = [0  , 1200 , 5000, 14000, float('inf')]
labels = ['Silver (<1200K)' ,  'Platine (1200K - 5K)' , 'Gold (5K - 14K)' , 'Diamante (> 14K )']

df_completo['Faixa_saldo'] = pd.cut(df_completo ['saldo'] , bins = bins , labels= labels)

distribuicao = df_completo['Faixa_saldo'].value_counts().reindex(labels)

df_completo.to_csv('data/clean_data', index= False, encoding='utf-8')

def plot_3_bar(distribuicao):
    fig, ax = plt.subplots(figsize=(10,6))
    sns.barplot(x=distribuicao.index, y=distribuicao.values, palette='viridis', ax=ax)
    plt.title("Distribuição por faixa de saldo")
    
    
    st.pyplot(fig) 

def plot_3_dif(dist):
    df_plot = dist.reset_index()
    df_plot.columns = ['Classificação de faixa', 'Quantidade de clientes']

    fig, ax = plt.subplots(figsize=(10,6))
    sns.set_theme(style='darkgrid')
    sns.pointplot(data=df_plot, x='Classificação de faixa', y='Quantidade de clientes', ax=ax)
    
    plt.title('Tendência de Volume por faixa de saldo')
    plt.xticks(rotation=15)
    
    st.pyplot(fig) 





    

## Agora é interessante trabalhar com machine learning, alimentando algum método para fazer previsões

### Um método a ser considerado é o Random Forest para fazer previsões sobre investimento

In [10]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score , confusion_matrix
import joblib

df_completo['Faixa_num'] = df_completo['Faixa_saldo'].cat.codes

x = df_completo[['saldo', 'Faixa_num']]
# Nesse caso o modelo vai prever um crescimento de 15%
y = df_completo['saldo'] * 1.15

x_train , x_test, y_train , y_test = train_test_split(x, y, test_size = 0.2 ,random_state= 42)

#Inicio do treino
rf_model = RandomForestRegressor(n_estimators = 100 , random_state= 42)
rf_model.fit (x_train , y_train)
# Avaliação
previsoes = rf_model.predict(x_test)
erro_medio = mean_absolute_error(y_test, previsoes)
score = r2_score(y_test, previsoes)

print (f" Avaliação do Modelo : ")
print (f" - Erro Médio Absoluto (MAE) : R$ {erro_medio:.2f}")
print (f"- Precisao (R ** 2 Score) : {score:.4f}")

#Salvando o metodo para nao ficar carregando todas as vezes no streamlit

joblib.dump(rf_model, 'modelo_investimento_rf.pkl')
print ("✅Modelo Salvo com sucesso")

 Avaliação do Modelo : 
 - Erro Médio Absoluto (MAE) : R$ 2.80
- Precisao (R ** 2 Score) : 1.0000
✅Modelo Salvo com sucesso
